In [2]:
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger
import numpy as np

In [3]:
# Размерность матрицы
n = 3
m = 5

C = np.array([
    [9, 4, 6, 5, 6],
    [17, 11, 15, 3, 7],
    [20, 9, 15, 7, 25]
])

i_s = ord('a')

# Переменные
X = np.array(list(LpVariable(f'{chr(i)}', lowBound=0, cat=LpInteger) for i in range(i_s, i_s + n * m)))\
    .reshape((n, m))

# Запасы
A = np.array([270, 390, 290])

# Потребности
B = np.array([150, 100, 250, 340, 110])


Проверим, задача является закрытого типа (сумма потребностей = сумме запасов)

In [4]:
A.sum() == B.sum()

True

In [3]:
# Целевая функция
F = (C * X).sum()

le = np.vectorize(lambda x, y: x <= y)
eq = np.vectorize(lambda x, y: x == y)

# Ограничения запасов
C1 = le(X.sum(axis=1), A)
# Ограничения потребностей
C2 = eq(X.sum(axis=0), B)

In [4]:
model = LpProblem("Distribution_Problem", LpMinimize)
model += F
for i in [C1, C2]:
    for j in i:
        model += j

In [5]:
model

Distribution_Problem:
MINIMIZE
9*a + 4*b + 6*c + 5*d + 6*e + 17*f + 11*g + 15*h + 3*i + 7*j + 20*k + 9*l + 15*m + 7*n + 25*o + 0
SUBJECT TO
_C1: a + b + c + d + e <= 270

_C2: f + g + h + i + j <= 390

_C3: k + l + m + n + o <= 290

_C4: a + f + k = 150

_C5: b + g + l = 100

_C6: c + h + m = 250

_C7: d + i + n = 340

_C8: e + j + o = 110

VARIABLES
0 <= a Integer
0 <= b Integer
0 <= c Integer
0 <= d Integer
0 <= e Integer
0 <= f Integer
0 <= g Integer
0 <= h Integer
0 <= i Integer
0 <= j Integer
0 <= k Integer
0 <= l Integer
0 <= m Integer
0 <= n Integer
0 <= o Integer

In [6]:
model.solve()

1

In [7]:
X_star = np.vectorize(lambda x: x.value())(np.array(model.variables()))

print('Оптимальный план перевозки:')
X_star.reshape((n, m))

Оптимальный план перевозки:


array([[150.,   0., 120.,   0.,   0.],
       [  0.,   0.,   0., 280., 110.],
       [  0., 100., 130.,  60.,   0.]])

In [8]:
print('Оптимальная стоимость перевозки')
model.objective.value()

Оптимальная стоимость перевозки


6950.0